# gamma 单谱

In [1]:
%jsroot on
TFile *ipf = new TFile("../MergedData/decay47_128.root");
TTree *tree = (TTree*)ipf->Get("tree");
TCanvas *c1 = new TCanvas;

In [2]:
TCut gtcut = "360-at<10000 && 360-at>-300";

In [3]:
tree->Draw("ae:decaytime>>(100,0,1000,800,0,800)",gtcut,"colz");
c1->SetLogz();
c1->Draw();

### 128Ag 寿命 ～ 60 ms

In [4]:
gROOT->ProcessLine(".L peaks.C");

In [5]:
int halflife = 60; // ms

In [6]:
tree->Draw("ae>>he_raw(3500,0,3500)",Form("decaytime<%d",4*halflife)) && gtcut;
tree->Draw("ae>>he_bgshort(3500,0,3500)",Form("decaytime>%d && decaytime<%d",4*halflife,8*halflife)) && gtcut;
tree->Draw("ae>>he_bglong(3500,0,3500)",Form("decaytime>4000 && decaytime<%d",4000+4*halflife)) && gtcut;
TH1D *he_raw = (TH1D*)gROOT->FindObject("he_raw");
TH1D *he_bgshort = (TH1D*)gROOT->FindObject("he_bgshort");
TH1D *he_bglong = (TH1D*)gROOT->FindObject("he_bglong");

In [7]:
TH1D *hbgsubshort = new TH1D("hbgsubshort","bgsubshort",3500,0,3500);
hbgsubshort->Add(he_raw,he_bgshort,1,-1);
TH1D *hbgsublong = new TH1D("hbgsublong","bgsublong",3500,0,3500);
hbgsublong->Add(he_raw,he_bglong,1,-1);

### 扣康普顿平台

以便于比较峰的统计

In [8]:
peaks("he_raw",0.05,1);
peaks("he_bgshort",0.05,1);
peaks("he_bglong",0.05,1);
peaks("hbgsubshort",0.05,1);
peaks("hbgsublong",0.05,1);

### 扣去较近的衰变时间窗

In [9]:
//%jsroot on
peaks("he_raw",0.01);
hbgsubshort->SetLineColor(kRed);
//peaks("hbgsubshort");
hbgsubshort->Draw("same");
c1->Draw();

### 扣去很远的衰变时间窗

In [10]:
he_raw->Draw();
hbgsublong->SetLineColor(kRed);
hbgsublong->Draw("same");
c1->Draw();

### ibeta cut

In [11]:
TH1D *h2;

In [12]:
tree->Draw("ae>>h1(1500,0,1500)","decaytime<450");
tree->Draw("ae>>h2(1500,0,1500)","decaytime<450 && ibeta<=2");
h1->Draw();
h2 = (TH1D*)gROOT->FindObject("h2");
h2->SetLineColor(kRed);
h2->Draw("same");
c1->Draw();

### 导出单谱数据

In [13]:
ofstream ofs("spectra/128Cd_13_47.txt");
for (int ibin=1; ibin<=2000; ibin++){
    ofs<<hbgsubshort->GetBinContent(ibin)<<'\n';
}
ofs.close();

### beta 事件数

In [14]:
tree->Draw("ibeta>>hnbeta(140,0,140)");
hnbeta->Draw();
c1->Draw();

In [15]:
for (int ibin=140; ibin>=2; ibin--){
    int hibin = hnbeta->GetBinContent(ibin);
    //cout<<ibin<<endl;
    for (int jbin=ibin-1; jbin>=2; jbin--){
        int hjbin = hnbeta->GetBinContent(jbin);
        hnbeta->SetBinContent(jbin, hjbin-hibin);
    }
}

In [16]:
hnbeta->Draw();
c1->SetLogy(0);
c1->Draw();